# Background

In this notebook, we will be analyzing the results of an AB test performed by the company, we will be carrying out the following steps:
we will be examining the performance in trial vs control stores to provide a recommendation for each location based on our insight. Below are some of the areas I want you to focus on, of course if you discover any other interesting insights feel free to include them in your findings.

-Select control stores – explore the data and define metrics for your control store selection – think about what would make them a control store. Look at the drivers and make sure you visualise these in a graph to better determine if they are suited. For this piece it may even be worth creating a function to help you.

-Assessment of the trial – this one should give you some interesting insights into each of the stores, check each trial store individually in comparison with the control store to get a clear view of its overall performance. We want to know if the trial stores were successful or not.

-Collate findings – summarise your findings for each store and provide an recommendation that we can share with Julia outlining the impact on sales during the trial period.

we will be considering the monthly sales experience of each store.

This can be broken down by:

- total sales revenue
- total number of customers
- average number of transactions per customer
- Create a measure to compare different control stores to each of the trial stores to do this write a function to reduce having to re-do the analysis for each trial store. Consider using Pearson correlations or a metric such as a magnitude distance e.g. 1- (Observed distance – minimum distance)/(Maximum distance – minimum distance) as a measure.

Once you have selected your control stores, compare each trial and control pair during the trial period. You want to test if total sales are significantly different in the trial period and if so, check if the driver of change is more purchasing customers or more purchases per customers etc.

In [1]:
import pandas as pd
import datetime
from datetime import datetime
import scipy
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
data = pd.read_csv('QVI_data.csv')

In [3]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [5]:
data.STORE_NBR.nunique()

272

We have 272 stores in our dataset.

In [6]:
data["Month_Id"] = pd.to_datetime(data['DATE'])

In [7]:
data['Month_Id'] = data['Month_Id'].dt.strftime('%Y%m')
data = data.drop(['DATE'],axis =1)

In [8]:
data.head()

,LYLTY_CARD_NBR,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,Month_Id
0,1000,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,201810
1,1002,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,201809
2,1003,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,201903
3,1003,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,201903
4,1004,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,201811


In [9]:
#grouping the data on by store and month. This is important because we will be aggregating and analyzing our data on a monthly level.
grouped_data = data.groupby(by = ['STORE_NBR', 'Month_Id'])

In [25]:
#purchase_summary = grouped_data.agg({'TOT_SALES': ['sum'], })

In [68]:
measureOverTime = (data.groupby(["STORE_NBR", "Month_Id"], as_index= False)
                      .agg({"TOT_SALES": "sum", 
                            "LYLTY_CARD_NBR": pd.Series.nunique, 
                            "TXN_ID": pd.Series.nunique, 
                            "PROD_QTY": "sum"})
                      .rename(columns={"LYLTY_CARD_NBR": "nCustomers", 
                                       "TXN_ID": "nTxnPerCust", 
                                       "PROD_QTY":"nChipsPerTxn"}))
measureOverTime["nTxnPerCust"] = measureOverTime["nTxnPerCust"]/measureOverTime["nCustomers"]
measureOverTime["avgPricePerUnit"] = measureOverTime["TOT_SALES"]/measureOverTime["nChipsPerTxn"]

In [69]:
measureOverTime

,STORE_NBR,Month_Id,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,1,201807,206.9,49,1.061224,62,3.337097
1,1,201808,176.1,42,1.023810,54,3.261111
2,1,201809,278.8,59,1.050847,75,3.717333
3,1,201810,188.1,44,1.022727,58,3.243103
4,1,201811,192.6,46,1.021739,57,3.378947
...,...,...,...,...,...,...,...
3164,272,201902,395.5,45,1.066667,91,4.346154
3165,272,201903,442.3,50,1.060000,101,4.379208
3166,272,201904,445.1,54,1.018519,105,4.239048
3167,272,201905,314.6,34,1.176471,71,4.430986


In [12]:
measureOverTime.columns

Index(['STORE_NBR', 'Month_Id', 'TOT_SALES', 'nCustomers', 'nTxnPerCust',
       'nChipsPerTxn', 'avgPricePerUnit'],
      dtype='object')

In [70]:
# First, create a new column in the DataFrame with store frequency
measureOverTime["store_frequency"] = measureOverTime.groupby("STORE_NBR")["STORE_NBR"].transform("count")

# Then, filter the DataFrame to only include stores with 12 observations (the full set of observations)
storesWithFullObs = measureOverTime[measureOverTime["store_frequency"] == 12]

# Finally, rename the columns of the filtered DataFrame
storesWithFullObs = storesWithFullObs.rename(columns={"STORE_NBR": "STORE_NBR", "store_frequency": "N"})

In [14]:
storesWithFullObs

,STORE_NBR,Month_Id,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,N
0,1,201807,206.9,49,1.061224,62,3.337097,12
1,1,201808,176.1,42,1.023810,54,3.261111,12
2,1,201809,278.8,59,1.050847,75,3.717333,12
3,1,201810,188.1,44,1.022727,58,3.243103,12
4,1,201811,192.6,46,1.021739,57,3.378947,12
...,...,...,...,...,...,...,...,...
3164,272,201902,395.5,45,1.066667,91,4.346154,12
3165,272,201903,442.3,50,1.060000,101,4.379208,12
3166,272,201904,445.1,54,1.018519,105,4.239048,12
3167,272,201905,314.6,34,1.176471,71,4.430986,12


In [15]:
preTrialMeasures = measureOverTime[(measureOverTime["Month_Id"] < '201902') & (measureOverTime["STORE_NBR"].isin(storesWithFullObs["STORE_NBR"]))]


In [16]:
preTrialMeasures.Month_Id.unique()

array(['201807', '201808', '201809', '201810', '201811', '201812',
       '201901'], dtype=object)

In [17]:
preTrialMeasures

,STORE_NBR,Month_Id,TOT_SALES,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,store_frequency
0,1,201807,206.9,49,1.061224,62,3.337097,12
1,1,201808,176.1,42,1.023810,54,3.261111,12
2,1,201809,278.8,59,1.050847,75,3.717333,12
3,1,201810,188.1,44,1.022727,58,3.243103,12
4,1,201811,192.6,46,1.021739,57,3.378947,12
...,...,...,...,...,...,...,...,...
3159,272,201809,304.7,32,1.125000,71,4.291549,12
3160,272,201810,430.6,44,1.136364,99,4.349495,12
3161,272,201811,376.2,41,1.097561,87,4.324138,12
3162,272,201812,403.9,47,1.000000,89,4.538202,12


In [18]:
#### Create a function to calculate correlation for a measure, looping through each control store.
#For Sales
trialStore_sales = preTrialMeasures[preTrialMeasures["STORE_NBR"] == 77]
trialStore_sales = trialStore_sales[["STORE_NBR","Month_Id","TOT_SALES","nCustomers"]]

def calCorr(preTrialMeasures, trialStore_sales, trialStoreN):
    calTable = pd.DataFrame(columns=["Store1", "Store2", "corr_measure"])
    stN = preTrialMeasures[["STORE_NBR"]]
    for i in stN["STORE_NBR"]:
        contSt = preTrialMeasures[preTrialMeasures["STORE_NBR"] == i]
        contSt = contSt[["TOT_SALES"]]
        core, pvalue = scipy.stats.pearsonr(trialStore_sales['TOT_SALES'],(contSt['TOT_SALES']))         
        #calMeasure = pd.DataFrame({"Store1": [trialStoreN], "Store2": [i], "corr_measure": [contSt["TOT_SALES"].corr(trialStore_sales["TOT_SALES"])]})
        calMeasure = pd.DataFrame({"Store1": trialStoreN, "Store2": i, "corr_measure": core},index=[0])
        #calTable = calTable.append(calMeasure)
        calTable = pd.concat([calTable, calMeasure])
        calTable = calTable.drop_duplicates()
    return calTable

# About CalCorr
The function takes preTrialMeasures, trialStore_sales, and trialStoreN as its input. Inside the function, it creates an empty dataframe 'calTable' with the desired columns. It then selects the column STORE_NBR from preTrialMeasures. The for loop iterates over the STORE_NBR of preTrialMeasures and for each store_NBR, it filters rows of preTrialMeasures with store_NBR and select 'totSales' column. Then it appends a new row to the 'calTable' dataframe with the values of store1, store2, and correlation measure between the totSales of trialStore_sales and preTrialMeasures for the current store_NBR. The function returns a table of each store in the data set and the correlation of each Store's Average sales with the specified trial store.  
I will define another function Cal_CorrCust below which will do the same as Cal_corrSales only it calculates the correlation for the number of customers for each store vs the trial  store.

In [19]:
def calculateCorrelation(preTrialMeasures, trialStore_sales, trialStoreN):
    calTable = pd.DataFrame(columns=["Store1", "Store2", "corr_measure"])
    stN = preTrialMeasures[["STORE_NBR"]]
    for i in stN["STORE_NBR"]:
        contSt = preTrialMeasures[preTrialMeasures["STORE_NBR"] == i]
        contSt = contSt[["nCustomers"]]
        corr_measure = contSt["nCustomers"].corr(trialStore_sales["nCustomers"])
        #calMeasure = pd.DataFrame({"Store1": [trialStoreN], "Store2": [i], "corr_measure": [corr_measure]})
        core, pvalue = scipy.stats.pearsonr(trialStore_sales['nCustomers'],(contSt['nCustomers']))         
        #calMeasure = pd.DataFrame({"Store1": [trialStoreN], "Store2": [i], "corr_measure": [contSt["TOT_SALES"].corr(trialStore_sales["TOT_SALES"])]})
        calMeasure = pd.DataFrame({"Store1": trialStoreN, "Store2": i, "corr_measure": core},index=[0])
        #calTable = calTable.append(calMeasure)
        calTable = pd.concat([calTable, calMeasure])
        calTable = calTable.drop_duplicates()

    return calTable

# Standardized Magnitude Distance
Next, we are going to define functions to calculate the magnitude distance between the trial and control stores, the first function, calculateMagnitudeDistance1 takes in the PretrialMeasures, trialStore_sales and trial store number. It then calculates the the absolute difference between the sales for the trial store and each store for each month.  
The second function takes the output from the first function and then calculates the magnitude distance (1- (Observed distance – minimum distance)/(Maximum distance – minimum distance) between the trial store and each control store.  
I will define the functions for both sales and number of customers.

In [20]:
#### Create a function to calculate a standardised magnitude distance for a measure, looping through each control store
##Sales
def calculateMagnitudeDistance1(preTrialMeasures, trialStore_sales, trial_storeN):
    calTable = pd.DataFrame(columns=["Store1", "Store2", "Month_Id", "mag_measure"])
    stN = preTrialMeasures[["STORE_NBR"]]
    for i in stN["STORE_NBR"]:
        contSt = preTrialMeasures[preTrialMeasures["STORE_NBR"] == i]
        contSt = contSt[["TOT_SALES"]]
        mag_measure = abs(trialStore_sales["TOT_SALES"].reset_index() - contSt["TOT_SALES"].reset_index())
        calMeasure = pd.DataFrame({"Store1": trial_storeN, "Store2": i, "Month_Id":preTrialMeasures["Month_Id"],"mag_measure":mag_measure['TOT_SALES']},index=[0])
        calTable = pd.concat([calTable, calMeasure])
        calTable = calTable.drop_duplicates()
    return calTable

import pandas as pd

def standMag1(magnitude_nSales):
    minMaxDist = magnitude_nSales.groupby(["Store1", "Month_Id"]).agg({"mag_measure": ["min", "max"]}).reset_index()
    minMaxDist.columns = ["Store1", "Month_Id", "minDist", "maxDist"]
    distTable = pd.merge(magnitude_nSales, minMaxDist, on=["Store1", "Month_Id"])
    distTable["magnitudeMeasure"] = 1 - (distTable["mag_measure"] - distTable["minDist"]) / (distTable["maxDist"] - distTable["minDist"])
    finalDistTable = distTable.groupby(["Store1", "Store2"])["magnitudeMeasure"].mean().reset_index()
    return finalDistTable


In [21]:
#Customers
def calculateMagnitudeDistance2(preTrialMeasures, trialStore_sales, trial_storeN):
    calTable = pd.DataFrame(columns=["Store1", "Store2", "Month_Id", "mag_measure"])
    stN = preTrialMeasures[["STORE_NBR"]]
    for i in stN["STORE_NBR"]:
        contSt = preTrialMeasures[preTrialMeasures["STORE_NBR"] == i]
        contSt = contSt[["nCustomers"]]
        mag_measure = abs(trialStore_sales["nCustomers"].reset_index() - contSt["nCustomers"].reset_index())
        calMeasure = pd.DataFrame({"Store1": trial_storeN, "Store2": i, "Month_Id":preTrialMeasures["Month_Id"],"mag_measure":mag_measure['nCustomers']},index=[0])
        calTable = pd.concat([calTable, calMeasure])
        calTable = calTable.drop_duplicates()
    return calTable
def standMag2(magnitude_nCustomers):
    minMaxDist = magnitude_nCustomers.groupby(["Store1", "Month_Id"]).agg({"mag_measure": ["min", "max"]}).reset_index()
    minMaxDist.columns = ["Store1", "Month_Id", "minDist", "maxDist"]
    distTable = pd.merge(magnitude_nCustomers, minMaxDist, on=["Store1", "Month_Id"])
    distTable["magnitudeMeasure"] = 1 - (distTable["mag_measure"] - distTable["minDist"]) / (distTable["maxDist"] - distTable["minDist"])
    finalDistTable = distTable.groupby(["Store1", "Store2"])["magnitudeMeasure"].mean().reset_index()
    return finalDistTable

In [22]:
magnitude_nCustomers = calculateMagnitudeDistance2(preTrialMeasures,trialStore_sales, trial_store)
magnitude_nCustomers

NameError: name 'trial_store' is not defined

In [23]:
trial_store = 77
corr_nSales = calCorr(preTrialMeasures,trialStore_sales,trial_store)
corr_nCustomers = calculateCorrelation(preTrialMeasures, trialStore_sales, trial_store )

# Use the functions for calculating magnitude
magnitude_nSales = calculateMagnitudeDistance1(preTrialMeasures, trialStore_sales, trial_store)
magnitude_nSales = standMag1(magnitude_nSales)
magnitude_nCustomers = calculateMagnitudeDistance2(preTrialMeasures,trialStore_sales, trial_store)
magnitude_nCustomers = standMag2(magnitude_nCustomers)

In [24]:
magnitude_nCustomers

,Store1,Store2,magnitudeMeasure
0,77,1,0.978947
1,77,2,0.873684
2,77,3,0.357895
3,77,4,0.189474
4,77,5,0.557895
...,...,...,...
255,77,268,0.968421
256,77,269,0.452632
257,77,270,0.526316
258,77,271,0.526316


I will  combine the all the scores calculated using our function to create a composite score to rank on. Let's take a simple average of the correlation and magnitude scores for each driver. Note that if we consider it more important for the trend of the drivers to be similar, we can increase the weight of the correlation score (a simple average gives a weight of 0.5 to the corr_weight) or if we consider the absolute size of the drivers to be more important, we can lower the weight of the correlation score.

In [25]:
corr_weight = 0.5
score_nSales = pd.merge(corr_nSales,magnitude_nSales, on=["Store1", "Store2"])
score_nSales['scoreNSales'] = (score_nSales['corr_measure'] * corr_weight)+(score_nSales['magnitudeMeasure'] * (1 - corr_weight))
score_nCustomers = pd.merge(corr_nCustomers,magnitude_nCustomers, on=["Store1", "Store2"])
score_nCustomers['scoreNCust'] = (score_nCustomers['corr_measure'] * corr_weight)+(score_nCustomers['magnitudeMeasure'] * (1 - corr_weight))


In [26]:
score_Control = pd.merge(score_nSales, score_nCustomers, on=["Store1", "Store2"])
score_Control["finalControlScore"] = score_Control["scoreNSales"] * 0.5 + score_Control["scoreNCust"] * 0.5

In [27]:
control_store = score_Control.sort_values(by='finalControlScore', ascending=False)
control_store = control_store['Store2']
control_store = control_store.iloc[1]

In [28]:
control_store

233

In [29]:
score_Control

,Store1,Store2,corr_measure_x,magnitudeMeasure_x,scoreNSales,corr_measure_y,magnitudeMeasure_y,scoreNCust,finalControlScore
0,77,1,0.075218,0.932426,0.503822,0.322168,0.978947,0.650558,0.57719
1,77,2,-0.263079,0.890259,0.31359,-0.572051,0.873684,0.150817,0.232203
2,77,3,0.806644,0.316822,0.561733,0.834207,0.357895,0.596051,0.578892
3,77,4,-0.2633,0.170851,-0.046224,-0.295639,0.189474,-0.053083,-0.049653
4,77,5,-0.110652,0.612748,0.251048,0.370659,0.557895,0.464277,0.357662
...,...,...,...,...,...,...,...,...,...
255,77,268,0.344757,0.945280,0.645018,0.369517,0.968421,0.668969,0.656994
256,77,269,-0.31573,0.484967,0.084618,-0.474293,0.452632,-0.01083,0.036894
257,77,270,0.31543,0.499399,0.407415,-0.131259,0.526316,0.197528,0.302471
258,77,271,0.355487,0.504059,0.429773,0.019629,0.526316,0.272972,0.351373


In [53]:
measureOverTimeSales = measureOverTime.copy()
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store, 'Trial',
                                              (np.where(measureOverTimeSales['STORE_NBR'] == control_store, 'Control', 'Other stores')))
pastSales = measureOverTimeSales.groupby(['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'].astype(str).str[:-2]+'-'+pastSales['Month_Id'].astype(str).str[-2:], format='%Y-%m')
#pastSales['TransactionMonth'] = pd.to_datetime((pastSales['YEARMONTH']//100).astype(str) + '-' + (pastSales['YEARMONTH']%100).astype(str) + '-01')

pastSales = pastSales[pastSales["Month_Id"] < '201903']

In [54]:
fig = px.line(pastSales, x="TransactionMonth", y="TOT_SALES", color = 'Store_type',title='Total Sales by Month')
fig.show()

# Trial Assessment
The trial period goes from the start of February 2019 to April 2019. We now want to see if there has been an uplift  in overall chip sales.We'll start with scaling the control store's sales to a level similar to control for any differences between the two stores outside of the trial period. Then we will calculate the percentage difference for the trial store and the control store during the period of the experiment. Finally we will try to see the difference is Statistically significance.

In [57]:
# Scale pre-trial control sales to match pre-trial trial store sales
preTrialMeasures = preTrialMeasures.copy()

scalingFactorForControlSales = preTrialMeasures.query('STORE_NBR == @trial_store and Month_Id < "201902"')['TOT_SALES'].sum() / preTrialMeasures.query('STORE_NBR == @control_store and Month_Id < "201902"')['TOT_SALES'].sum()

#Applying the Scaling Factor
measureOverTimeSales = measureOverTime.copy()
scaledControlSales = measureOverTimeSales.query('STORE_NBR == @control_store')
scaledControlSales = scaledControlSales.copy()
scaledControlSales['controlSales'] = scaledControlSales['TOT_SALES'] * scalingFactorForControlSales


In [58]:
preTrialMeasures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1820 entries, 0 to 3163
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STORE_NBR        1820 non-null   int64  
 1   Month_Id         1820 non-null   object 
 2   TOT_SALES        1820 non-null   float64
 3   nCustomers       1820 non-null   int64  
 4   nTxnPerCust      1820 non-null   float64
 5   nChipsPerTxn     1820 non-null   int64  
 6   avgPricePerUnit  1820 non-null   float64
 7   store_frequency  1820 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 192.5+ KB


Now we have comparable sales figures for the control store, we can calculate the percentage difference between the scaled control sales and the trial store's sales during the trial period. We will then check if the difference is statistically significant.

In [59]:
#measureOverTime = pd.DataFrame(measureOverTime)
percentageDiff = pd.merge(scaledControlSales[["Month_Id", "controlSales"]], measureOverTime[measureOverTime["STORE_NBR"] == trial_store][["TOT_SALES", "Month_Id"]], on = "Month_Id")
percentageDiff["percentageDiff"] = abs(percentageDiff["controlSales"] - percentageDiff["TOT_SALES"]) / percentageDiff["controlSales"]
#Here, a new DataFrame is created from measureOverTime, then it creates a new DataFrame called percentageDiff by merging the scaledControlSales table and the measureOverTime table (filtered to only include the store number specified in the variable trial_store) on the "Month_Id" column. Finally, it creates a new column in the percentageDiff table called "percentageDiff" which calculates the absolute difference between the "controlSales" and "totSales" columns divided by the "controlSales" column.


We will test with a null hypothesis of there being 0 difference between trial and control stores. We will Calculate the t-values for the trial months. After that, find the 95th percentile of the t distribution with the appropriate degrees of freedomto check whether the hypothesis is statistically significant.

In [60]:
#Calculate the standard deviation of the scaled percentage difference in the pre-trial period
stdDev = np.std(percentageDiff['Month_Id'] < '201902')


# there are 8 months in the pre-trial period
# hence 8 - 1 = 7 degrees of freedom
#Set the degrees of freedom to 7
degreesOfFreedom = 7

#Calculate the t-values for the trial months
trial_months = percentageDiff[(percentageDiff['Month_Id'] < '201905') & (percentageDiff['Month_Id'] > '201901')]
#df1 = df[(df.a != -1) & (df.b != -1)]
trial_months = trial_months.copy()
trial_months['tvalue'] = (trial_months['percentageDiff'] - 0)/stdDev
trial_months = trial_months.copy()
trial_months['TransactionMonth'] = pd.to_datetime(trial_months['Month_Id'], format='%Y%m')

#Find the 95th percentile of the t-distribution with the appropriate degrees of freedom
from scipy.stats import t
t.ppf(0.95, df = degreesOfFreedom)

1.894578605061305

In [71]:
measureOverTimeSales = pd.DataFrame(measureOverTime)#(Note)

#Assign store type to each store
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeSales['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastSales = measureOverTimeSales.groupby(by=['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()

#Convert year-month to transaction month
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'], format='%Y%m')

#Select only trial and control store
pastSales = pastSales[pastSales['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastSales_Controls95 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls95['TOT_SALES'] = pastSales_Controls95['TOT_SALES'] * (1 + stdDev * 2)
pastSales_Controls95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastSales_Controls5 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls5['TOT_SALES'] = pastSales_Controls5['TOT_SALES'] * (1 - stdDev * 2)
pastSales_Controls5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessment = pd.concat([pastSales,pastSales_Controls95,pastSales_Controls5])
trialAssessment1 = trialAssessment[(trialAssessment['Month_Id'] < '201905') & (trialAssessment['Month_Id'] > '201901')]


#Visualize using plotly
fig = px.line(trialAssessment, x='TransactionMonth', y='TOT_SALES', color='Store_type')
fig.show()


In [67]:
pastSales_Controls5

,Month_Id,Store_type,TOT_SALES,TransactionMonth
0,201807,Control 5th % confidence,4.065935,2018-07-01
3,201808,Control 5th % confidence,3.998798,2018-08-01
6,201809,Control 5th % confidence,3.197360,2018-09-01
9,201810,Control 5th % confidence,2.597331,2018-10-01
12,201811,Control 5th % confidence,2.959586,2018-11-01
15,201812,Control 5th % confidence,3.913479,2018-12-01
18,201901,Control 5th % confidence,2.482640,2019-01-01
21,201902,Control 5th % confidence,3.412755,2019-02-01
24,201903,Control 5th % confidence,2.784753,2019-03-01
27,201904,Control 5th % confidence,2.218291,2019-04-01


In [ ]:
#### As our null hypothesis is that the trial period is the same as the pre-trial period, let's take the standard deviation based on the scaled percentage difference in the pre-trial period 
stdDev <- sd(percentageDiff[YEARMONTH < 201902 , percentageDiff]) 

degreesOfFreedom <- 7

#### Trial and control store number of customers
measureOverTimeCusts <- as.data.table(measureOverTime)
pastCustomers <- measureOverTimeCusts[, Store_type := ifelse(STORE_NBR == trial_store, "Trial", ifelse(STORE_NBR == control_store, "Control", "Other stores"))][, nCusts := mean(nCustomers), by = c("YEARMONTH","Store_type")][, TransactionMonth := as.Date(paste(YEARMONTH %/% 100, YEARMONTH %% 100, 1, sep = "‐"), "%Y‐%m‐%d")
][Store_type %in% c("Trial", "Control"), ]

###Control 95th percentile
pastCustomers_Control95 <- pastCustomers[Store_type == "Control",][, nCusts := nCusts * (1 + stdDev * 2)][, Store_type := "Control 95th % confidence interval"]

###Control 5th percentile
pastCustomers_Control5 <- pastCustomers[Store_type == "Control",][, nCusts := nCusts * (1 + stdDev * 2)][, Store_type := "Control 5th % confidence interval"]

trialAssessment <- rbind(pastCustomers,pastCustomers_Control95,pastCustomers_Control5)

###Visualize
ggplot(trialAssessment, aes(TransactionMonth, nCusts, color = Store_type)) + geom_rect(data = trialAssessment[YEARMONTH < 201905 & YEARMONTH > 201901 , ], aes(xmin = min(TransactionMonth), xmax =  max(TransactionMonth), ymin = 0, ymax = Inf, coor = NULL), show.legend = F) + geom_line(aes(linetype = Store_type)) + labs(x = "Month Of Operation", y = "Total Number of Customers", title = "Total Number of Customers by Month")

In [72]:
#Assign store type to each store
measureOverTimeCusts['Store_type'] = np.where(measureOverTimeCusts['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeCusts['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastCustomers = measureOverTimeCusts.groupby(by=['Month_Id','Store_type'])['nCustomers'].mean().reset_index()

#Convert year-month to transaction month
pastCustomers['TransactionMonth'] = pd.to_datetime(pastCustomers['Month_Id'], format='%Y%m')

#Select only trial and control store
pastCustomers = pastCustomers[pastCustomers['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastCustomers_Control95 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control95['nCustomers'] = pastCustomers_Control95['nCustomers'] * (1 + stdDev * 2)
pastCustomers_Control95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastCustomers_Control5 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control5['nCustomers'] = pastCustomers_Control5['nCustomers'] * (1 - stdDev * 2)
pastCustomers_Control5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessmentC = pd.concat([pastCustomers,pastCustomers_Control95,pastCustomers_Control5])

#Visualize using plotly
fig = px.line(trialAssessmentC, x='TransactionMonth', y='nCustomers', color='Store_type')
fig.show()


In [78]:
pastCustomers_Control95

,Month_Id,Store_type,nCustomers,TransactionMonth
0,201807,Control,51.0,2018-07-01
3,201808,Control,48.0,2018-08-01
6,201809,Control,42.0,2018-09-01
9,201810,Control,35.0,2018-10-01
12,201811,Control,40.0,2018-11-01
15,201812,Control,47.0,2018-12-01
18,201901,Control,35.0,2019-01-01
21,201902,Control,45.0,2019-02-01
24,201903,Control,40.0,2019-03-01
27,201904,Control,30.0,2019-04-01


# Trial Store 86

We will repeat all of the above steps for trial store 86, we will find the control store and assess the impact of the test on the stores.

In [92]:
trial_store = 86
trialStore_sales = preTrialMeasures[preTrialMeasures["STORE_NBR"] == 86]
trialStore_sales = trialStore_sales[["STORE_NBR","Month_Id","TOT_SALES","nCustomers"]]
#trialStore_sales = measureOverTime[measureOverTime['STORE_NBR'] == trial_store][['STORE_NBR','Month_Id','TOT_SALES','nCustomers']]
corr_nSales = calCorr(preTrialMeasures,trialStore_sales,trial_store)
corr_nCustomers = calculateCorrelation(preTrialMeasures, trialStore_sales, trial_store )

# Use the functions for calculating magnitude
magnitude_nSales = calculateMagnitudeDistance1(preTrialMeasures, trialStore_sales, trial_store)
magnitude_nSales = standMag1(magnitude_nSales)
magnitude_nCustomers = calculateMagnitudeDistance2(preTrialMeasures,trialStore_sales, trial_store)
magnitude_nCustomers = standMag2(magnitude_nCustomers)

corr_weight = 0.5
score_nSales = pd.merge(corr_nSales,magnitude_nSales, on=["Store1", "Store2"])
score_nSales['scoreNSales'] = (score_nSales['corr_measure'] * corr_weight)+(score_nSales['magnitudeMeasure'] * (1 - corr_weight))
score_nCustomers = pd.merge(corr_nCustomers,magnitude_nCustomers, on=["Store1", "Store2"])
score_nCustomers['scoreNCust'] = (score_nCustomers['corr_measure'] * corr_weight)+(score_nCustomers['magnitudeMeasure'] * (1 - corr_weight))

score_Control = pd.merge(score_nSales, score_nCustomers, on=["Store1", "Store2"])
score_Control["finalControlScore"] = score_Control["scoreNSales"] * 0.5 + score_Control["scoreNCust"] * 0.5
control_store = score_Control.sort_values(by='finalControlScore', ascending=False)
control_store = control_store['Store2']
control_store = control_store.iloc[1]
control_store

155

We have found store 155 as the control store for store 86, Next we will see if the factors Total sales and Number of customers are the same for both before the trial.

In [93]:
measureOverTimeSales = measureOverTime.copy()
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store, 'Trial',
                                              np.where(measureOverTimeSales['STORE_NBR'] == control_store, 'Control', 'Other stores'))
pastSales = measureOverTimeSales.groupby(['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'].astype(str).str[:-2]+'-'+pastSales['Month_Id'].astype(str).str[-2:], format='%Y-%m')
#pastSales['TransactionMonth'] = pd.to_datetime((pastSales['YEARMONTH']//100).astype(str) + '-' + (pastSales['YEARMONTH']%100).astype(str) + '-01')

pastSales = pastSales[pastSales["Month_Id"] < '201903']


fig = px.line(pastSales, x="TransactionMonth", y="TOT_SALES", color = 'Store_type',title='Total Sales by Month')
fig.show()


In [96]:
measureOverTimeCusts = measureOverTime.copy()
measureOverTimeCusts['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store, 'Trial',
                                              np.where(measureOverTimeSales['STORE_NBR'] == control_store, 'Control', 'Other stores'))
pastCustomers = measureOverTimeCusts.groupby(['Month_Id','Store_type'])['nCustomers'].mean().reset_index()
pastCustomers['TransactionMonth'] = pd.to_datetime(pastCustomers['Month_Id'].astype(str).str[:-2]+'-'+pastSales['Month_Id'].astype(str).str[-2:], format='%Y-%m')
#pastSales['TransactionMonth'] = pd.to_datetime((pastSales['YEARMONTH']//100).astype(str) + '-' + (pastSales['YEARMONTH']%100).astype(str) + '-01')

pastCustomers = pastCustomers[pastCustomers["Month_Id"] < '201903']


fig = px.line(pastCustomers, x="TransactionMonth", y="nCustomers", color = 'Store_type',title='Number of Customers')
fig.show()


In [91]:
#Observations

77

# Trial Assessment
We will also assess the impact of the trial for the two stores.

In [98]:
# Scale pre-trial control sales to match pre-trial trial store sales
preTrialMeasures = preTrialMeasures.copy()

scalingFactorForControlSales = preTrialMeasures.query('STORE_NBR == @trial_store and Month_Id < "201902"')['TOT_SALES'].sum() / preTrialMeasures.query('STORE_NBR == @control_store and Month_Id < "201902"')['TOT_SALES'].sum()

#Applying the Scaling Factor
measureOverTimeSales = measureOverTime.copy()
scaledControlSales = measureOverTimeSales.query('STORE_NBR == @control_store')
scaledControlSales = scaledControlSales.copy()
scaledControlSales['controlSales'] = scaledControlSales['TOT_SALES'] * scalingFactorForControlSales

measureOverTime = pd.DataFrame(measureOverTime)
percentageDiff = pd.merge(scaledControlSales[["Month_Id", "controlSales"]], measureOverTime[measureOverTime["STORE_NBR"] == trial_store][["TOT_SALES", "Month_Id"]], on = "Month_Id")
percentageDiff["percentageDiff"] = abs(percentageDiff["controlSales"] - percentageDiff["TOT_SALES"]) / percentageDiff["controlSales"]
stdDev = np.std(percentageDiff['Month_Id'] < '201902')

# there are 8 months in the pre-trial period
# hence 8 - 1 = 7 degrees of freedom
#Set the degrees of freedom to 7
degreesOfFreedom = 7

#Calculate the t-values for the trial months
trial_months = percentageDiff[(percentageDiff['Month_Id'] < '201905') & (percentageDiff['Month_Id'] > '201901')]
#df1 = df[(df.a != -1) & (df.b != -1)]
trial_months = trial_months.copy()
trial_months['tvalue'] = (trial_months['percentageDiff'] - 0)/stdDev
trial_months = trial_months.copy()
trial_months['TransactionMonth'] = pd.to_datetime(trial_months['Month_Id'], format='%Y%m')

#Find the 95th percentile of the t-distribution with the appropriate degrees of freedom
from scipy.stats import t
t.ppf(0.95, df = degreesOfFreedom)

1.894578605061305

In [100]:
control_store

155

In [101]:
##measureOverTimeSales = pd.DataFrame(measureOverTime)(Note)

#Assign store type to each store
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeSales['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastSales = measureOverTimeSales.groupby(by=['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()

#Convert year-month to transaction month
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'], format='%Y%m')

#Select only trial and control store
pastSales = pastSales[pastSales['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastSales_Controls95 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls95['TOT_SALES'] = pastSales_Controls95['TOT_SALES'] * (1 + stdDev * 2)
pastSales_Controls95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastSales_Controls5 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls5['TOT_SALES'] = pastSales_Controls5['TOT_SALES'] * (1 - stdDev * 2)
pastSales_Controls5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessment = pd.concat([pastSales,pastSales_Controls95,pastSales_Controls5])

#Visualize using plotly
fig = px.line(trialAssessment, x='TransactionMonth', y='TOT_SALES', color='Store_type')
fig.show()


In [103]:
#### As our null hypothesis is that the trial period is the same as the pre-trial period, let's take the standard deviation based on the scaled percentage difference in the pre-trial period 
stdDev <- sd(percentageDiff[YEARMONTH < 201902 , percentageDiff]) 

degreesOfFreedom <- 7

#### Trial and control store number of customers
measureOverTimeCusts <- as.data.table(measureOverTime)
pastCustomers <- measureOverTimeCusts[, Store_type := ifelse(STORE_NBR == trial_store, "Trial", ifelse(STORE_NBR == control_store, "Control", "Other stores"))][, nCusts := mean(nCustomers), by = c("YEARMONTH","Store_type")][, TransactionMonth := as.Date(paste(YEARMONTH %/% 100, YEARMONTH %% 100, 1, sep = "‐"), "%Y‐%m‐%d")
][Store_type %in% c("Trial", "Control"), ]

###Control 95th percentile
pastCustomers_Control95 <- pastCustomers[Store_type == "Control",][, nCusts := nCusts * (1 + stdDev * 2)][, Store_type := "Control 95th % confidence interval"]

###Control 5th percentile
pastCustomers_Control5 <- pastCustomers[Store_type == "Control",][, nCusts := nCusts * (1 + stdDev * 2)][, Store_type := "Control 5th % confidence interval"]

trialAssessment <- rbind(pastCustomers,pastCustomers_Control95,pastCustomers_Control5)

###Visualize
ggplot(trialAssessment, aes(TransactionMonth, nCusts, color = Store_type)) + geom_rect(data = trialAssessment[YEARMONTH < 201905 & YEARMONTH > 201901 , ], aes(xmin = min(TransactionMonth), xmax =  max(TransactionMonth), ymin = 0, ymax = Inf, coor = NULL), show.legend = F) + geom_line(aes(linetype = Store_type)) + labs(x = "Month Of Operation", y = "Total Number of Customers", title = "Total Number of Customers by Month")

SyntaxError: invalid syntax (1503745332.py, line 7)

In [102]:
#Assign store type to each store
measureOverTimeCusts['Store_type'] = np.where(measureOverTimeCusts['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeCusts['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastCustomers = measureOverTimeCusts.groupby(by=['Month_Id','Store_type'])['nCustomers'].mean().reset_index()

#Convert year-month to transaction month
pastCustomers['TransactionMonth'] = pd.to_datetime(pastCustomers['Month_Id'], format='%Y%m')

#Select only trial and control store
pastCustomers = pastCustomers[pastCustomers['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastCustomers_Control95 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control95['nCustomers'] = pastCustomers_Control95['nCustomers'] * (1 + stdDev * 2)
pastCustomers_Control95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastCustomers_Control5 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control5['nCustomers'] = pastCustomers_Control5['nCustomers'] * (1 - stdDev * 2)
pastCustomers_Control5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessmentC = pd.concat([pastCustomers,pastCustomers_Control95,pastCustomers_Control5])

#Visualize using plotly
fig = px.line(trialAssessmentC, x='TransactionMonth', y='nCustomers', color='Store_type')
fig.show()


# Trial Store88

In [103]:
trial_store = 88
trialStore_sales = preTrialMeasures[preTrialMeasures["STORE_NBR"] == 88]
trialStore_sales = trialStore_sales[["STORE_NBR","Month_Id","TOT_SALES","nCustomers"]]
#trialStore_sales = measureOverTime[measureOverTime['STORE_NBR'] == trial_store][['STORE_NBR','Month_Id','TOT_SALES','nCustomers']]
corr_nSales = calCorr(preTrialMeasures,trialStore_sales,trial_store)
corr_nCustomers = calculateCorrelation(preTrialMeasures, trialStore_sales, trial_store )

# Use the functions for calculating magnitude
magnitude_nSales = calculateMagnitudeDistance1(preTrialMeasures, trialStore_sales, trial_store)
magnitude_nSales = standMag1(magnitude_nSales)
magnitude_nCustomers = calculateMagnitudeDistance2(preTrialMeasures,trialStore_sales, trial_store)
magnitude_nCustomers = standMag2(magnitude_nCustomers)

corr_weight = 0.5
score_nSales = pd.merge(corr_nSales,magnitude_nSales, on=["Store1", "Store2"])
score_nSales['scoreNSales'] = (score_nSales['corr_measure'] * corr_weight)+(score_nSales['magnitudeMeasure'] * (1 - corr_weight))
score_nCustomers = pd.merge(corr_nCustomers,magnitude_nCustomers, on=["Store1", "Store2"])
score_nCustomers['scoreNCust'] = (score_nCustomers['corr_measure'] * corr_weight)+(score_nCustomers['magnitudeMeasure'] * (1 - corr_weight))

score_Control = pd.merge(score_nSales, score_nCustomers, on=["Store1", "Store2"])
score_Control["finalControlScore"] = score_Control["scoreNSales"] * 0.5 + score_Control["scoreNCust"] * 0.5
control_store = score_Control.sort_values(by='finalControlScore', ascending=False)
control_store = control_store['Store2']
control_store = control_store.iloc[1]
control_store

178

In [104]:
measureOverTimeSales = measureOverTime.copy()
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store, 'Trial',
                                              np.where(measureOverTimeSales['STORE_NBR'] == control_store, 'Control', 'Other stores'))
pastSales = measureOverTimeSales.groupby(['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'].astype(str).str[:-2]+'-'+pastSales['Month_Id'].astype(str).str[-2:], format='%Y-%m')
#pastSales['TransactionMonth'] = pd.to_datetime((pastSales['YEARMONTH']//100).astype(str) + '-' + (pastSales['YEARMONTH']%100).astype(str) + '-01')

pastSales = pastSales[pastSales["Month_Id"] < '201903']


fig = px.line(pastSales, x="TransactionMonth", y="TOT_SALES", color = 'Store_type',title='Total Sales by Month')
fig.show()

In [107]:
measureOverTimeCusts = pd.DataFrame(measureOverTime)
pastCustomers = measureOverTimeCusts.assign(Store_type = lambda x: np.where(x.STORE_NBR == trial_store, 'Trial',
                                                                       np.where(x.STORE_NBR == control_store, 'Control', 'Other stores')))

In [108]:
pastCustomers = pastCustomers.groupby(['Month_Id','Store_type'])['nCustomers'].transform('mean')
pastCustomers['TransactionMonth'] = pd.to_datetime(pastCustomers['Month_Id'],format='%Y%m')
pastCustomers = pastCustomers[pastCustomers['Month_Id'] < '201903']
fig = px.line(pastCustomers, x="TransactionMonth", y="nCustomers", color = 'Store_type',title='Total Sales by Month')
fig.show()

KeyError: 'Month_Id'

In [109]:
# Scale pre-trial control sales to match pre-trial trial store sales
preTrialMeasures = preTrialMeasures.copy()

scalingFactorForControlSales = preTrialMeasures.query('STORE_NBR == @trial_store and Month_Id < "201902"')['TOT_SALES'].sum() / preTrialMeasures.query('STORE_NBR == @control_store and Month_Id < "201902"')['TOT_SALES'].sum()

#Applying the Scaling Factor
measureOverTimeSales = measureOverTime.copy()
scaledControlSales = measureOverTimeSales.query('STORE_NBR == @control_store')
scaledControlSales = scaledControlSales.copy()
scaledControlSales['controlSales'] = scaledControlSales['TOT_SALES'] * scalingFactorForControlSales


#Now we have comparable sales figures for the control store, we can calculate the percentage difference between the scaled control sales and the trial store's sales during the trial period. We will then check if the difference is statistically significant.

measureOverTime = pd.DataFrame(measureOverTime)
percentageDiff = pd.merge(scaledControlSales[["Month_Id", "controlSales"]], measureOverTime[measureOverTime["STORE_NBR"] == trial_store][["TOT_SALES", "Month_Id"]], on = "Month_Id")
percentageDiff["percentageDiff"] = abs(percentageDiff["controlSales"] - percentageDiff["TOT_SALES"]) / percentageDiff["controlSales"]
#Here, a new DataFrame is created from measureOverTime, then it creates a new DataFrame called percentageDiff by merging the scaledControlSales table and the measureOverTime table (filtered to only include the store number specified in the variable trial_store) on the "Month_Id" column. Finally, it creates a new column in the percentageDiff table called "percentageDiff" which calculates the absolute difference between the "controlSales" and "totSales" columns divided by the "controlSales" column.


#We will test with a null hypothesis of there being 0 difference between trial and control stores. We will Calculate the t-values for the trial months. After that, find the 95th percentile of the t distribution with the appropriate degrees of freedomto check whether the hypothesis is statistically significant.

#Calculate the standard deviation of the scaled percentage difference in the pre-trial period
stdDev = np.std(percentageDiff['Month_Id'] < '201902')


# there are 8 months in the pre-trial period
# hence 8 - 1 = 7 degrees of freedom
#Set the degrees of freedom to 7
degreesOfFreedom = 7

#Calculate the t-values for the trial months
trial_months = percentageDiff[(percentageDiff['Month_Id'] < '201905') & (percentageDiff['Month_Id'] > '201901')]
#df1 = df[(df.a != -1) & (df.b != -1)]
trial_months = trial_months.copy()
trial_months['tvalue'] = (trial_months['percentageDiff'] - 0)/stdDev
trial_months = trial_months.copy()
trial_months['TransactionMonth'] = pd.to_datetime(trial_months['Month_Id'], format='%Y%m')

#Find the 95th percentile of the t-distribution with the appropriate degrees of freedom
from scipy.stats import t
t.ppf(0.95, df = degreesOfFreedom)

1.894578605061305

In [109]:
##measureOverTimeSales = pd.DataFrame(measureOverTime)(Note)

#Assign store type to each store
measureOverTimeSales['Store_type'] = np.where(measureOverTimeSales['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeSales['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastSales = measureOverTimeSales.groupby(by=['Month_Id','Store_type'])['TOT_SALES'].mean().reset_index()

#Convert year-month to transaction month
pastSales['TransactionMonth'] = pd.to_datetime(pastSales['Month_Id'], format='%Y%m')

#Select only trial and control store
pastSales = pastSales[pastSales['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastSales_Controls95 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls95['TOT_SALES'] = pastSales_Controls95['TOT_SALES'] * (1 + stdDev * 2)
pastSales_Controls95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastSales_Controls5 = pastSales[pastSales['Store_type'] == 'Control'].copy()
pastSales_Controls5['TOT_SALES'] = pastSales_Controls5['TOT_SALES'] * (1 - stdDev * 2)
pastSales_Controls5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessment = pd.concat([pastSales,pastSales_Controls95,pastSales_Controls5])

#Visualize using plotly
fig = px.line(trialAssessment, x='TransactionMonth', y='TOT_SALES', color='Store_type')
fig.show()


In [110]:
#Assign store type to each store
measureOverTimeCusts['Store_type'] = np.where(measureOverTimeCusts['STORE_NBR'] == trial_store,'Trial',np.where(measureOverTimeCusts['STORE_NBR'] == control_store,'Control','Other stores'))

#group by store type and year-month and calculate the mean of sales
pastCustomers = measureOverTimeCusts.groupby(by=['Month_Id','Store_type'])['nCustomers'].mean().reset_index()

#Convert year-month to transaction month
pastCustomers['TransactionMonth'] = pd.to_datetime(pastCustomers['Month_Id'], format='%Y%m')

#Select only trial and control store
pastCustomers = pastCustomers[pastCustomers['Store_type'].isin(['Trial','Control'])]

#Control Store 95th percentile
pastCustomers_Control95 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control95['nCustomers'] = pastCustomers_Control95['nCustomers'] * (1 + stdDev * 2)
pastCustomers_Control95['Store_type'] = 'Control 95th % confidence'

#Control Store 5th percentile
pastCustomers_Control5 = pastCustomers[pastCustomers['Store_type'] == 'Control'].copy()
pastCustomers_Control5['nCustomers'] = pastCustomers_Control5['nCustomers'] * (1 - stdDev * 2)
pastCustomers_Control5['Store_type'] = 'Control 5th % confidence'

#Join all the dataframe
trialAssessmentC = pd.concat([pastCustomers,pastCustomers_Control95,pastCustomers_Control5])

#Visualize using plotly
fig = px.line(trialAssessmentC, x='TransactionMonth', y='nCustomers', color='Store_type')
fig.show()


# Final Conclusion